# Species Detection Exploration
This will cover:

Top detected species by device.  
Top detected species cumulatively.  
Top detected species by week/month per device.  
Top detected species by week/month overall.  

# Setup System Path

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd


# Go up one level to .../audiomoth
PROJECT_ROOT = Path(os.getcwd()).resolve().parent

# Add project root to sys.path so `src` is importable
sys.path.insert(0, str(PROJECT_ROOT))

PROCESSED_DATA_PATH = out_dir = (
    Path(PROJECT_ROOT) / "data_processed" / "analysis_df.parquet"
)
analysis_df = pd.read_parquet(PROCESSED_DATA_PATH)

# Make pandas show more columns/rows while exploring
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

## Top detected species


In [ ]:
SPECIES_COL = "common_name"

TOP_N = 20

top_species_overall = (
    analysis_df.groupby(SPECIES_COL)
    .size()
    .sort_values(ascending=False)
    .head(TOP_N)
    .rename("detections")
    .reset_index()
)


total = len(analysis_df)
top_species_overall["%"] = (top_species_overall["detections"] / total * 100).round(2)
top_species_overall

## Top Detected Species By Device

In [ ]:
TOP_N = 10

# For each device, list the species in order of most detections
top_species_per_device = (
    analysis_df.groupby(["device", SPECIES_COL])
    .size()
    .rename("detections")
    .reset_index()
    .sort_values(["device", "detections"], ascending=[True, False])
)

# Add a column that ranks species within each device
top_species_per_device["rank_within_device"] = top_species_per_device.groupby("device")[
    "detections"
].rank(method="first", ascending=False)

# Keep only the top N ranked species per device
top_species_per_device = top_species_per_device.loc[
    top_species_per_device["rank_within_device"] <= TOP_N
]

# Pivot to wide format for easier viewing
top_species_per_device_wide = top_species_per_device.pivot_table(
    index="device", columns="rank_within_device", values=SPECIES_COL, aggfunc="first"
)

top_species_per_device_wide

## To Detected Species Per Week

### Overall

In [ ]:
TOP_N = 5

species_by_week_overall = (
    analysis_df.groupby(["week", SPECIES_COL])
    .size()
    .rename("detections")
    .reset_index()
    .sort_values(["week", "detections"], ascending=[True, False])
)

species_by_week_overall["rank_within_week"] = species_by_week_overall.groupby("week")[
    "detections"
].rank(method="first", ascending=False)

species_by_week_overall = species_by_week_overall.loc[
    species_by_week_overall["rank_within_week"] <= TOP_N
]
species_by_week_overall.head(30)

In [ ]:
out_dir = Path(PROJECT_ROOT) / "outputs"
out_dir.mkdir(parents=True, exist_ok=True)

species_by_week_overall.to_csv(out_dir / "top_species_by_week.csv", index=False)

### Per Device

In [ ]:
TOP_N = 10

device_week_species = (
    analysis_df.groupby(["device", "week", SPECIES_COL])
    .size()
    .rename("detections")
    .reset_index()
    .sort_values(["device", "week", "detections"], ascending=[True, True, False])
    .reset_index(drop=True)
)

device_week_species["rank"] = device_week_species.groupby(["device", "week"])[
    "detections"
].rank(method="first", ascending=False)

device_week_species = device_week_species.loc[device_week_species["rank"] <= TOP_N]
device_week_species.head(10)

In [ ]:
out_dir = Path(PROJECT_ROOT) / "outputs"
out_dir.mkdir(parents=True, exist_ok=True)

device_week_species.to_csv(out_dir / "device_top_species_by_week.csv", index=False)

In [ ]:
def device_week_table(df: pd.DataFrame, device: str) -> pd.DataFrame:
    subset = df.loc[df["device"] == device].copy()

    wide = subset.pivot_table(
        index="week", columns="rank", values=[SPECIES_COL], aggfunc="first"
    )

    # Flatten column names
    wide.columns = [f"{int(rank)}" for val, rank in wide.columns]

    return wide.sort_index()

In [ ]:
device_tables = {
    device: device_week_table(device_week_species, device)
    for device in device_week_species["device"].unique()
}

device_tables["CWT2"]

In [ ]:
out_dir = Path(PROJECT_ROOT) / "outputs" / "device_week_tables"
out_dir.mkdir(parents=True, exist_ok=True)


for device, table in device_tables.items():
    table.to_csv(out_dir / f"{device}_top_species_by_week.csv")